In [1]:
import os
import sys
import time
import random
import warnings
import collections
from multiprocessing import Pool
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, Lasso, BayesianRidge, ElasticNet

sys.path.append('../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

### -----DECLARE EVERYTHING HERE-----

In [4]:
# configuration
mco_id = 2
date = '2021_01_14'
use_cache = False
#use_cache = True

mco_name = 'uhc' if mco_id == 2 else 'agp' if mco_id == 1 else 'fail'
auth_table_name = mco_name + '_healthstar_authorizations_' + date
apts_table_name = mco_name + '_healthstar_appointments_' + date
input_batch_descr = mco_name + ' healthstar auths and appts delta'
auth_descr = mco_name + ' auths delta'
apts_descr = mco_name + ' appts delta'
source = 'hs_' + mco_name
con = cb_utils.get_engine()
seed = random.randint(0, 100)

print(f'Seed: {seed}')
random

Seed: 3


<module 'random' from '/usr/lib/python3.8/random.py'>

### GET AUTHS

In [5]:
query = f"select max(ath.watermark) highwatermark from cb.hcbs_auths ath where ath.mco_id = {mco_id}"
auth_watermark = cb_utils.sql_query_to_df(query, use_cache=use_cache)
hcbs_auth_highwatermark = -1 #auth_watermark['highwatermark'][0]
hcbs_auth_highwatermark

Pulling query from db


-1

In [6]:
query = f"select * from visitverify.v_analytics_auths2 vath where vath.watermark > {hcbs_auth_highwatermark}"
auths = cb_utils.sql_query_to_df(query, use_cache=use_cache, source=source)

Pulling query from db


In [7]:
auths.shape

(227439, 41)

### GET APTS

In [8]:
query = f"select max(watermark) highwatermark from cb.appointments_visits av where av.mco_id = {mco_id}"
apt_auth_watermark = cb_utils.sql_query_to_df(query, use_cache=use_cache)
apt_highwatermark = -1 #apt_auth_watermark['highwatermark'][0]
apt_highwatermark

Pulling query from db


-1

In [ ]:
query = f"select * from visitverify.v_analytics_appointments2 vapt where vapt.watermark > {apt_highwatermark}"
apts = cb_utils.sql_query_to_df(query, use_cache=use_cache, source=source)

Pulling query from db


In [ ]:
apts.shape

### CREATE batch_input_id

In [ ]:
conn = cb_utils.get_conn()
query = """
insert into cb.input_batches (mco_id, description)
select %s, %s
returning id;
"""
with conn.cursor() as cur:
    cur.execute(query, [ mco_id, input_batch_descr])
    result = cur.fetchone()
conn.commit()
conn.close()

batch_id = result[0]
batch_id

### INSERT AUTHS INTO RAW

In [ ]:
conn = cb_utils.get_conn()
query = """
insert into cb.inputs (input_batch_id, file_name, description, type, table_name, mco_id)
select %s, %s, %s, 'auth_hcbs', %s, %s
returning id;
"""
with conn.cursor() as cur:
    cur.execute(query, [ batch_id, auth_table_name, auth_descr, auth_table_name, mco_id])
    result = cur.fetchone()
conn.commit()
conn.close()

auth_input_id = result[0]
auth_input_id

In [ ]:
auths.to_sql(auth_table_name, con, 'raw')

In [ ]:
query = f"alter table raw.{auth_table_name} add column raw_id bigserial; select 1 x;"
cb_utils.sql_query_to_df(query, use_cache=use_cache)

### INSERT APTS INTO RAW

In [ ]:
conn = cb_utils.get_conn()
query = """
insert into cb.inputs (input_batch_id, file_name, description, type, table_name, mco_id)
select %s, %s, %s, 'hcbs_visits', %s, %s
returning id;
"""
with conn.cursor() as cur:
    cur.execute(query, [ batch_id, apts_table_name, apts_descr, apts_table_name, mco_id])
    result = cur.fetchone()
conn.commit()
conn.close()

apt_input_id = result[0]
apt_input_id

In [ ]:
apts.to_sql(apts_table_name, con, 'raw')

In [ ]:
query = f"alter table raw.{apts_table_name} add column raw_id bigserial; select 1 x;"
cb_utils.sql_query_to_df(query, use_cache=use_cache)